In [43]:
# !pip install catboost
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold , StratifiedKFold,RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.impute import KNNImputer , SimpleImputer

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
import torch

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)


# Program to generate a random number between 0 and 9

# importing the random module
import random


In [44]:
def compute_pfbeta(labels, predictions, beta=1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
            #cfp += 1 - prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp+1e-8)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [45]:
import cv2

In [46]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
import torch.nn as nn
import torch.nn.functional as F

In [85]:
data = pd.read_csv('output/oof_test_data_all.csv')
# test_data = pd.read_csv

In [86]:
data_last_fold = pd.read_csv('output/oof_test_data.csv')

In [87]:
target = ["cancer"]
# features = ['site_id', 'patient_id', 'image_id', 'laterality', 'view', 'age','implant', 'machine_id']
# list_numeric_features = data._get_numeric_data().columns.drop(target)
# list_cat_features = list(set(data[features].columns) - set(list_numeric_features))

## Feature Engineering

## Model Building 

In [88]:
df = data.copy()

In [89]:
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,orientation,orignal_path,dump_path,kfold,mean_age,weights,fold_0_pred,fold_1_pred,fold_2_pred,fold_3_pred
0,2,14951,1296262783,1,1,59.0,0,0,0,NaN,0,NaN,21,False,R-CC,input_data/train_images/14951/1296262783.dcm,input_data/train_crop_voilut_2048/14951/129626...,1,60.198516,1,0.001573,0.024738,0.000262,0.000119
1,1,21520,1542713553,0,0,43.0,0,0,0,1.0,0,C,216,False,L-MLO,input_data/train_images/21520/1542713553.dcm,input_data/train_crop_voilut_2048/21520/154271...,1,59.490566,1,0.003296,0.021013,0.005217,0.000350
2,2,3872,909567878,1,1,52.0,0,0,0,NaN,0,NaN,21,False,R-CC,input_data/train_images/3872/909567878.dcm,input_data/train_crop_voilut_2048/3872/9095678...,3,60.198516,1,0.000376,0.169066,0.000896,0.014158
3,1,52479,750902630,0,0,55.0,0,0,0,1.0,0,C,49,False,L-MLO,input_data/train_images/52479/750902630.dcm,input_data/train_crop_voilut_2048/52479/750902...,4,57.268687,1,0.003055,0.029604,0.001778,0.000269
4,2,12047,404122403,0,0,62.0,0,0,0,NaN,0,NaN,48,False,L-MLO,input_data/train_images/12047/404122403.dcm,input_data/train_crop_voilut_2048/12047/404122...,2,60.435452,1,0.005471,0.019081,0.008757,0.001270


In [90]:
df.columns

Index(['site_id', 'patient_id', 'image_id', 'laterality', 'view', 'age',
       'cancer', 'biopsy', 'invasive', 'BIRADS', 'implant', 'density',
       'machine_id', 'difficult_negative_case', 'orientation', 'orignal_path',
       'dump_path', 'kfold', 'mean_age', 'weights', 'fold_0_pred',
       'fold_1_pred', 'fold_2_pred', 'fold_3_pred'],
      dtype='object')

In [91]:
test_data = pd.read_csv('input_data/test.csv')

In [92]:
test_data

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [93]:
ml_cols = ['patient_id','image_id', 'laterality','view', 'age','implant','machine_id',
       'cancer','kfold',
        'fold_0_pred','fold_1_pred', 'fold_2_pred', 'fold_3_pred','site_id']

In [94]:
df = df[ml_cols]

In [95]:
df.head()

,patient_id,image_id,laterality,view,age,implant,machine_id,cancer,kfold,fold_0_pred,fold_1_pred,fold_2_pred,fold_3_pred,site_id
0,14951,1296262783,1,1,59.0,0,21,0,1,0.001573,0.024738,0.000262,0.000119,2
1,21520,1542713553,0,0,43.0,0,216,0,1,0.003296,0.021013,0.005217,0.000350,1
2,3872,909567878,1,1,52.0,0,21,0,3,0.000376,0.169066,0.000896,0.014158,2
3,52479,750902630,0,0,55.0,0,49,0,4,0.003055,0.029604,0.001778,0.000269,1
4,12047,404122403,0,0,62.0,0,48,0,2,0.005471,0.019081,0.008757,0.001270,2


In [96]:
df['mean_preds'] = df[['fold_0_pred', 'fold_1_pred', 'fold_2_pred','fold_3_pred']].mean(axis=1)
df['max_preds'] = df[['fold_0_pred', 'fold_1_pred', 'fold_2_pred','fold_3_pred']].max(axis=1)
df['std_preds'] = df[['fold_0_pred', 'fold_1_pred', 'fold_2_pred','fold_3_pred']].std(axis=1)


In [97]:
# Create dummy variables for the 'color' column
# dummies = pd.get_dummies(df['view'],prefix='view')

# # Concatenate the dummy variables with the original DataFrame
# df = pd.concat([df, dummies], axis=1)

In [98]:
df

,patient_id,image_id,laterality,view,age,implant,machine_id,cancer,kfold,fold_0_pred,fold_1_pred,fold_2_pred,fold_3_pred,site_id,mean_preds,max_preds,std_preds
0,14951,1296262783,1,1,59.0,0,21,0,1,0.001573,0.024738,0.000262,0.000119,2,0.006673,0.024738,0.012061
1,21520,1542713553,0,0,43.0,0,216,0,1,0.003296,0.021013,0.005217,0.000350,1,0.007469,0.021013,0.009248
2,3872,909567878,1,1,52.0,0,21,0,3,0.000376,0.169066,0.000896,0.014158,2,0.046124,0.169066,0.082209
3,52479,750902630,0,0,55.0,0,49,0,4,0.003055,0.029604,0.001778,0.000269,1,0.008676,0.029604,0.013998
4,12047,404122403,0,0,62.0,0,48,0,2,0.005471,0.019081,0.008757,0.001270,2,0.008645,0.019081,0.007603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55857,7036,1085643986,0,1,50.0,0,29,0,2,0.000493,0.013822,0.000194,0.000257,2,0.003692,0.013822,0.006755
55858,8218,1080728445,0,0,61.0,0,49,0,0,0.024301,0.966083,0.020900,0.008854,1,0.255034,0.966083,0.474079
55859,45132,1249943162,0,0,62.0,0,29,0,2,0.021471,0.061270,0.007057,0.005226,2,0.023756,0.061270,0.026043
55860,51982,760601666,0,0,60.0,0,49,0,2,0.137308,0.077497,0.008443,0.020142,1,0.060847,0.137308,0.059236


In [99]:
master_df = df.groupby(['patient_id','laterality']).agg({'fold_0_pred':['max','mean','std'],
                                             'fold_1_pred':['max','mean','std'],
                                             'fold_2_pred':['max','mean','std'],
                                             'fold_3_pred':['max','mean','std'],
                                             'mean_preds' : ['max','mean','std'],
                                             'max_preds':['max','mean','std'],
                                            'std_preds':['max','mean','std'],
                                             'age':'mean',
                                             'implant':['count','max'],
                                             'machine_id':'max',
                                             'view':['max','nunique','min'],
                                             # 'view_1':'sum',
                                             # 'view_2':'sum',
                                             # 'view_3':'sum',
                                             # 'view_4':'sum',
                                             'site_id':'max',
                                             'kfold':'max',
                                             'cancer':['mean','max'],
                                                    
                                                        }).reset_index()
                                             

In [100]:
master_df.columns = master_df.columns.map('_'.join)

In [101]:
master_df['cancer_t'] = np.where(master_df.cancer_mean >= 0.5,1,0) 
master_df['cancer_t_max'] = np.where(master_df.cancer_max >= 0.5,1,0) 


In [102]:
master_df.head()

,patient_id_,laterality_,fold_0_pred_max,fold_0_pred_mean,fold_0_pred_std,fold_1_pred_max,fold_1_pred_mean,fold_1_pred_std,fold_2_pred_max,fold_2_pred_mean,fold_2_pred_std,fold_3_pred_max,fold_3_pred_mean,fold_3_pred_std,mean_preds_max,mean_preds_mean,mean_preds_std,max_preds_max,max_preds_mean,max_preds_std,std_preds_max,std_preds_mean,std_preds_std,age_mean,implant_count,implant_max,machine_id_max,view_max,view_nunique,view_min,site_id_max,kfold_max,cancer_mean,cancer_max,cancer_t,cancer_t_max
0,5,0,0.015035,0.012023,0.004259,0.924249,0.487678,0.617405,0.015358,0.013740,0.002287,0.001436,0.000854,0.000822,0.237223,0.128574,0.153653,0.924249,0.487678,0.617405,0.458059,0.239827,0.308627,68.0,2,0,21,1,2,0,2,1,0.0,0,0,0
1,5,1,0.016390,0.014938,0.002053,0.068315,0.067709,0.000857,0.050922,0.039703,0.015865,0.005076,0.004174,0.001275,0.034450,0.031631,0.003986,0.068315,0.067709,0.000857,0.030114,0.028820,0.001830,68.0,2,0,21,1,2,0,2,1,0.0,0,0,0
2,25,0,0.006450,0.005728,0.001021,0.035862,0.033599,0.003201,0.002547,0.001686,0.001218,0.004396,0.003020,0.001946,0.012314,0.011008,0.001846,0.035862,0.033599,0.003201,0.015779,0.015157,0.000880,62.0,2,0,49,1,2,0,1,0,0.0,0,0,0
3,25,1,0.163971,0.053223,0.073957,0.096409,0.056387,0.028612,0.014291,0.006222,0.005836,0.044540,0.012565,0.021330,0.061221,0.032099,0.020110,0.163971,0.089970,0.054172,0.070243,0.039750,0.023256,62.0,4,0,49,1,2,0,1,0,0.0,0,0,0
4,28,0,0.001524,0.001450,0.000105,0.962968,0.504787,0.647966,0.019220,0.011142,0.011424,0.000909,0.000650,0.000367,0.246155,0.129507,0.164965,0.962968,0.504787,0.647966,0.477951,0.250238,0.322034,40.0,2,0,49,1,2,0,1,4,0.0,0,0,0


In [103]:
master_df.cancer_max.value_counts()

0    22854
1      972
Name: cancer_max, dtype: int64

In [104]:
cc = master_df.copy()

In [87]:
cc['cancer_t'] = np.where(cc.cancer_mean >= 0.5,1,0) 

In [104]:
cc['cancer_p'] = np.where(cc.mean_preds_mean >= 0.5,1,0) 

In [105]:
cc['cancer_t'].value_counts()

0    22913
1      913
Name: cancer_t, dtype: int64

In [106]:
cc['cancer_p'].value_counts()

0    22916
1      910
Name: cancer_p, dtype: int64

In [109]:
compute_pfbeta(cc['cancer_t'], np.where(cc.mean_preds_max >= 0.5,1,0) )

0.828846153842169

In [285]:
compute_pfbeta(cc['cancer_t'], cc.mean_preds_mean)

0.4389047835802836

In [77]:
compute_pfbeta(cc['cancer_max'], cc['cancer_p'])

0.8611500701222013

In [55]:
master_df.cancer_mean.value_counts()

0.000000    22854
1.000000      492
0.500000      354
0.600000       48
0.400000       41
0.428571        9
0.571429        8
0.666667        6
0.333333        5
0.444444        4
0.555556        3
0.625000        2
Name: cancer_mean, dtype: int64

In [67]:
master_df.max_preds_max.value_counts()

1.000000    332
1.000000     92
1.000000     28
1.000000     19
0.999999      8
           ... 
0.022914      1
0.378647      1
0.021573      1
0.199662      1
0.024350      1
Name: max_preds_max, Length: 23242, dtype: int64

## ML

In [255]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [256]:
df_test_ml = master_df[master_df.kfold_max==4].reset_index(drop=True)
df_ml = master_df[master_df.kfold_max!=4].reset_index(drop=True)

In [257]:
df_test_ml.columns

Index(['patient_id_', 'laterality_', 'fold_0_pred_max', 'fold_0_pred_mean',
       'fold_0_pred_std', 'fold_1_pred_max', 'fold_1_pred_mean',
       'fold_1_pred_std', 'fold_2_pred_max', 'fold_2_pred_mean',
       'fold_2_pred_std', 'fold_3_pred_max', 'fold_3_pred_mean',
       'fold_3_pred_std', 'mean_preds_max', 'mean_preds_mean',
       'mean_preds_std', 'max_preds_max', 'max_preds_mean', 'max_preds_std',
       'std_preds_max', 'std_preds_mean', 'std_preds_std', 'age_mean',
       'implant_count', 'implant_max', 'machine_id_max', 'view_max',
       'view_nunique', 'view_min', 'site_id_max', 'kfold_max', 'cancer_mean',
       'cancer_max', 'cancer_t'],
      dtype='object')

In [258]:
target = ['cancer_t']
ml_train_cols = ['laterality_', 'fold_0_pred_max', 'fold_0_pred_mean',
       'fold_0_pred_std', 'fold_1_pred_max', 'fold_1_pred_mean',
       'fold_1_pred_std', 'fold_2_pred_max', 'fold_2_pred_mean',
       'fold_2_pred_std', 'fold_3_pred_max', 'fold_3_pred_mean',
       'fold_3_pred_std', 'mean_preds_max', 'mean_preds_mean',
       'mean_preds_std', 'max_preds_max', 'max_preds_mean', 'max_preds_std',
       'std_preds_max', 'std_preds_mean', 'std_preds_std', 'age_mean',
       'implant_count', 'implant_max', 'machine_id_max', 'view_max', 'view_min',
       'view_nunique','site_id_max']

In [259]:
col_indx_names = ['laterality_','machine_id_max', 'site_id_max','view_max','view_min']
cate_features_index = np.array([df_ml.columns.get_loc(col)-1 for col in col_indx_names])

In [260]:
cate_features_index

array([ 0, 25, 29, 26, 28])

In [261]:
df_ml[ml_train_cols].head()

,laterality_,fold_0_pred_max,fold_0_pred_mean,fold_0_pred_std,fold_1_pred_max,fold_1_pred_mean,fold_1_pred_std,fold_2_pred_max,fold_2_pred_mean,fold_2_pred_std,fold_3_pred_max,fold_3_pred_mean,fold_3_pred_std,mean_preds_max,mean_preds_mean,mean_preds_std,max_preds_max,max_preds_mean,max_preds_std,std_preds_max,std_preds_mean,std_preds_std,age_mean,implant_count,implant_max,machine_id_max,view_max,view_min,view_nunique,site_id_max
0,0,0.015035,0.012023,0.004259,0.924249,0.487678,0.617405,0.015358,0.013740,0.002287,0.001436,0.000854,0.000822,0.237223,0.128574,0.153653,0.924249,0.487678,0.617405,0.458059,0.239827,0.308627,68.0,2,0,21,1,0,2,2
1,1,0.016390,0.014938,0.002053,0.068315,0.067709,0.000857,0.050922,0.039703,0.015865,0.005076,0.004174,0.001275,0.034450,0.031631,0.003986,0.068315,0.067709,0.000857,0.030114,0.028820,0.001830,68.0,2,0,21,1,0,2,2
2,0,0.006450,0.005728,0.001021,0.035862,0.033599,0.003201,0.002547,0.001686,0.001218,0.004396,0.003020,0.001946,0.012314,0.011008,0.001846,0.035862,0.033599,0.003201,0.015779,0.015157,0.000880,62.0,2,0,49,1,0,2,1
3,1,0.163971,0.053223,0.073957,0.096409,0.056387,0.028612,0.014291,0.006222,0.005836,0.044540,0.012565,0.021330,0.061221,0.032099,0.020110,0.163971,0.089970,0.054172,0.070243,0.039750,0.023256,62.0,4,0,49,1,0,2,1
4,0,0.118059,0.071903,0.065275,0.093374,0.072442,0.029602,0.013875,0.010332,0.005011,0.005905,0.004159,0.002470,0.056930,0.039709,0.024355,0.118059,0.084785,0.047057,0.057419,0.039412,0.025467,54.0,2,0,21,1,0,2,2


In [262]:
df_ml[ml_train_cols].shape

(19060, 30)

In [263]:

lgb_params = { 
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc',
              'learning_rate' : 0.03,
              'num_boost_round':2000,
#               'max_depth' : 12, 
                'seed': 7,
                'verbose': 100
               }

xg_params = {'train_method':'hist','nthread':4,'eval_metric':'auc','eta':0.05,'seed':1993,'objective':'binary:logistic','grow_policy':'lossguide'} 



In [272]:
y_total=[]
p = []

for fold in df_ml.kfold_max.unique():
    
    X_train_k, X_test_k = df_ml[df_ml.kfold_max==fold][ml_train_cols], df_ml[df_ml.kfold_max!=fold][ml_train_cols]
    y_train_k, y_test_k =df_ml[df_ml.kfold_max==fold][target], df_ml[df_ml.kfold_max!=fold][target]
    
    model = CatBoostClassifier(n_estimators=2500,random_state=1993,max_depth=12,bagging_temperature=0.85,
                               eval_metric='PRAUC',use_best_model=True)
    
    # model = XGBClassifier(**xg_params)
    
    # model = LGBMClassifier(**lgb_params)
    
    model.fit(X_train_k,y_train_k,eval_set=[(X_test_k, y_test_k)], early_stopping_rounds=300,verbose=100)
    predicted = model.predict_proba(X_test_k)[:,1]
    print("roc_auc_score: ",metrics.roc_auc_score(y_test_k, predicted))
    p = model.predict_proba(df_test_ml[ml_train_cols])[:,1]
    y_total.append(p)

Learning rate set to 0.031293
0:	learn: 0.9651416	test: 0.9069520	best: 0.9069520 (0)	total: 40ms	remaining: 1m 39s
100:	learn: 1.0000000	test: 0.9769869	best: 0.9858311 (8)	total: 6.45s	remaining: 2m 33s
200:	learn: 1.0000000	test: 0.9770044	best: 0.9858311 (8)	total: 12.5s	remaining: 2m 22s
300:	learn: 1.0000000	test: 0.9770293	best: 0.9858311 (8)	total: 18.4s	remaining: 2m 14s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.985831145
bestIteration = 8

Shrink model to first 9 iterations.
roc_auc_score:  0.9994526654770702
Learning rate set to 0.031267
0:	learn: 0.9576825	test: 0.8298796	best: 0.8298796 (0)	total: 53.9ms	remaining: 2m 14s
100:	learn: 1.0000000	test: 0.9833860	best: 0.9869123 (19)	total: 6.36s	remaining: 2m 31s
200:	learn: 1.0000000	test: 0.9823798	best: 0.9869123 (19)	total: 12.7s	remaining: 2m 25s
300:	learn: 1.0000000	test: 0.9820599	best: 0.9869123 (19)	total: 19.3s	remaining: 2m 21s
Stopped by overfitting detector  (300 iterations wait)

best

In [273]:
### Find the variable Importance 

sorted(zip(model.feature_importances_,df_ml[ml_train_cols]),reverse=True)  

[(16.63356863328791, 'mean_preds_mean'),
 (7.543451406221384, 'fold_0_pred_mean'),
 (6.1216775849465686, 'fold_2_pred_mean'),
 (5.413482708649789, 'machine_id_max'),
 (4.7867381153840185, 'fold_1_pred_std'),
 (4.63094906099266, 'fold_3_pred_max'),
 (4.1820904941958466, 'max_preds_max'),
 (4.09821046484829, 'fold_3_pred_mean'),
 (3.8481008513146993, 'fold_2_pred_std'),
 (3.713781153765915, 'max_preds_std'),
 (3.606506934978123, 'mean_preds_max'),
 (3.4464171621595217, 'fold_2_pred_max'),
 (3.3350591194097303, 'std_preds_std'),
 (3.305192150739634, 'fold_0_pred_std'),
 (3.1341076943278687, 'fold_0_pred_max'),
 (3.0853057790386846, 'std_preds_max'),
 (3.0575767173041695, 'implant_count'),
 (2.9932886677961323, 'fold_3_pred_std'),
 (2.158185443382087, 'std_preds_mean'),
 (2.0341081446847538, 'fold_1_pred_mean'),
 (2.033379813702978, 'mean_preds_std'),
 (1.802936323413415, 'max_preds_mean'),
 (1.5757942496251818, 'age_mean'),
 (1.4500463761781328, 'implant_max'),
 (0.9728558926619612, 'fold

In [274]:
df_test_ml['cancer_p'] = np.mean(y_total,axis=0)

In [275]:
# df_test_ml['cancer_p'] = (df_test_ml['cancer_p'] > 0.3).astype(float)

In [281]:
compute_pfbeta(df_test_ml['cancer_t'], np.where(df_test_ml['cancer_p']>0.5,1,0))

0.7450980391913367

In [282]:
compute_pfbeta(df_test_ml['cancer_t'], df_test_ml['cancer_p'])

0.2941997272082176

In [277]:
metrics.confusion_matrix(df_test_ml['cancer_t'], np.where(df_test_ml['cancer_p']>0.5,1,0))

array([[4574,    8],
       [  70,  114]])

In [231]:
metrics.f1_score(df_test_ml['cancer_t'], np.where(df_test_ml['cancer_p']>0.5,1,0))

0.7368421052631579

In [224]:
import pickle

In [278]:
# Save the model to a file using pickle
with open('output/catboost_model.pickle', 'wb') as f:
    pickle.dump(model, f)

### Load model and predict on the test data 

In [8]:
class RSNADataSet(torch.utils.data.Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df
#         self.path = image_path
        self.transforms = transforms

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, item):
        
#         image = cv2.imread(self.df.iloc[item][self.path])
        d = self.df.iloc[item]
        image = cv2.imread(f'{png_dir}/{d.machine_id}/{d.patient_id}/{d.image_id}.png')
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
            
        return {
            "image": image,
#             "image_id": torch.as_tensor(self.df.iloc[item]['image_id']),
#             "meta":torch.as_tensor(self.df.iloc[item][config.CATEGORY_AUX_TARGETS])
                }


class RSNADataloader:
    def __init__(self, df, weights, transforms=None, test=False):
        self.df = df
#         self.path = image_path
        self.transforms = transforms
        self.weights = weights
        self.dataset = RSNADataSet(
            df=self.df,transforms=self.transforms)

    def fetch(self, batch_size, num_workers, drop_last=False, shuffle=False):
        sampler = torch.utils.data.SequentialSampler(self.dataset)
        data_loader = torch.utils.data.DataLoader(
        self.dataset, batch_size=batch_size, sampler=sampler, shuffle=shuffle, num_workers=num_workers, drop_last=drop_last)        
        return data_loader
    
    
class RSNAModel(torch.nn.Module):
    def __init__(self, output_size, n_cols):
        super().__init__()
        self.no_columns, self.output_size = n_cols, output_size
        self.register_buffer('mean', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))
        self.register_buffer('std', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))
        self.features = config.MODEL
        self.classification = nn.Linear(1792,1)
        
        
    def forward(self, image, meta=None,target=None,image_id=None):   
        
        # Image CNN
        x = image
#         batch_size,C,H,W = x.shape
#         x = (x - self.mean) / self.std
        # image = self.features.extract_features(image.float())
        features = self.features.forward_features(x.float())
        x = F.adaptive_avg_pool2d(features,1)
        x = torch.flatten(x,1,3)

        
        # CLASSIF
        out = self.classification(x)
        
        return torch.sigmoid(out)
   



In [9]:
def inference(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
         with amp.autocast(enabled=True):
            for data in tqdm(dataloader, total=len(dataloader)):
                for k, v in data.items():
                    data[k] = v.to(device)
                    inputs = data['image']
                    
                outputs = model(inputs)
                # 2 tta
                outputs += model(torch.flip(inputs, dims=[3]))
                outputs /= 2.0

#                 output = model(**data)

                output = outputs.detach().cpu().numpy()
                predictions.append(output)
            predictions = np.concatenate(predictions, axis=0)

    return predictions

def get_preds_folds(dataframe):
    
    data_loader =  RSNADataloader(dataframe,weights=None, transforms=config.VAL_TRANSFORM, test=True).fetch(
            batch_size=config.BATCH_SIZE, num_workers=config.NUM_WORKERS, shuffle=False, drop_last=False)
      
    model = RSNAModel(output_size = 1,n_cols=len(config.CATEGORY_AUX_TARGETS)).cuda()
    
    raw_preds = []

    for idx , model_filename in enumerate(config.MODEL_PATHS): ## nfolds = config.NFOLDS = 5
        
        print(f"{model_filename}+>>> inference")
        model.cuda();
        model.load_state_dict(torch.load(model_filename,map_location=torch.device('cuda')))
        print("model loaded successfully")
        predictions = inference(model, data_loader,config.DEVICE)
        raw_preds.append(predictions)


        del predictions #, pred_logits
        torch.cuda.empty_cache()
    
    return raw_preds


In [10]:
png_dir = 'input_data/train_VOI_LUT_1536'
model_paths = ['output/weighted_run/checkpoint-fold-0_ddp_ep_1_model.bin',
               'output/weighted_run/checkpoint-fold-1_ddp_ep_0_model.bin',
               'output/weighted_run/checkpoint-fold-2_ddp_ep_0_model.bin',
               'output/weighted_run/checkpoint-fold-3_ddp_ep_1_model.bin',
               'output/weighted_run/checkpoint-fold-4_ddp_ep_0_model.bin']


In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm.models.efficientnet import _cfg,efficientnet_b4
import torch.cuda.amp as amp

In [12]:
class config:
    MODEL_BACKBONE = 'efficientnet_b4'
    MODEL = efficientnet_b4(pretrained=True,num_classes=1)
    SEED = 2023
    BATCH_SIZE = 64
    NUM_WORKERS = 4
    FP16 = True 
    FP16_OPT_LEVEL = "O1"
    DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    NUM_CLASSES = 1
    IMG_SIZE = (1024, 768)
    CATEGORY_AUX_TARGETS =  ['site_id', 'laterality', 'view', 'implant', 'machine_id', 'age']
    TARGET = 'cancer'
    CLIP_THRES = 0.3
    
    # Data transformations
    
    MODEL_PATHS = ['output/weighted_run/checkpoint-fold-0_ddp_ep_1_model.bin',
               'output/weighted_run/checkpoint-fold-1_ddp_ep_0_model.bin',
               'output/weighted_run/checkpoint-fold-2_ddp_ep_0_model.bin',
               'output/weighted_run/checkpoint-fold-3_ddp_ep_1_model.bin',
               'output/weighted_run/checkpoint-fold-4_ddp_ep_0_model.bin']
    
    VAL_TRANSFORM = A.Compose([
                                A.Resize(IMG_SIZE[0],IMG_SIZE[1]),
                                # A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
                                ToTensorV2()
                            ])



In [13]:
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,orientation,orignal_path,dump_path,kfold,mean_age
0,2,10006,462822612,0,1,61.0,0,0,0,NaN,0,NaN,29,False,L-CC,input_data/train_images/10006/462822612.dcm,input_data/train_images_png/10006/462822612.png,3,59.153744
1,2,10006,1459541791,0,0,61.0,0,0,0,NaN,0,NaN,29,False,L-MLO,input_data/train_images/10006/1459541791.dcm,input_data/train_images_png/10006/1459541791.png,3,59.153744
2,2,10006,1864590858,1,0,61.0,0,0,0,NaN,0,NaN,29,False,R-MLO,input_data/train_images/10006/1864590858.dcm,input_data/train_images_png/10006/1864590858.png,3,59.153744
3,2,10006,1874946579,1,1,61.0,0,0,0,NaN,0,NaN,29,False,R-CC,input_data/train_images/10006/1874946579.dcm,input_data/train_images_png/10006/1874946579.png,3,59.153744
4,2,10011,220375232,0,1,55.0,0,0,0,0.0,0,NaN,21,True,L-CC,input_data/train_images/10011/220375232.dcm,input_data/train_images_png/10011/220375232.png,1,60.198516


In [14]:
predictions = get_preds_folds(df)

output/weighted_run/checkpoint-fold-0_ddp_ep_1_model.bin+>>> inference
model loaded successfully


100%|██████████| 855/855 [32:12<00:00,  2.26s/it]


output/weighted_run/checkpoint-fold-1_ddp_ep_0_model.bin+>>> inference
model loaded successfully


100%|██████████| 855/855 [32:32<00:00,  2.28s/it]


output/weighted_run/checkpoint-fold-2_ddp_ep_0_model.bin+>>> inference
model loaded successfully


100%|██████████| 855/855 [32:07<00:00,  2.25s/it]


output/weighted_run/checkpoint-fold-3_ddp_ep_1_model.bin+>>> inference
model loaded successfully


100%|██████████| 855/855 [32:32<00:00,  2.28s/it]


output/weighted_run/checkpoint-fold-4_ddp_ep_0_model.bin+>>> inference
model loaded successfully


100%|██████████| 855/855 [32:55<00:00,  2.31s/it]


In [18]:
for i in range(len(predictions)):
    df[f'fold_{i}_predictions'] = predictions[i]

In [26]:
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,orientation,orignal_path,dump_path,kfold,mean_age,fold_0_predictions,fold_1_predictions,fold_2_predictions,fold_3_predictions,fold_4_predictions
0,2,10006,462822612,0,1,61.0,0,0,0,NaN,0,NaN,29,False,L-CC,input_data/train_images/10006/462822612.dcm,input_data/train_images_png/10006/462822612.png,3,59.153744,0.008331,0.043457,0.045410,0.050110,0.116577
1,2,10006,1459541791,0,0,61.0,0,0,0,NaN,0,NaN,29,False,L-MLO,input_data/train_images/10006/1459541791.dcm,input_data/train_images_png/10006/1459541791.png,3,59.153744,0.040100,0.050537,0.046143,0.019897,0.031403
2,2,10006,1864590858,1,0,61.0,0,0,0,NaN,0,NaN,29,False,R-MLO,input_data/train_images/10006/1864590858.dcm,input_data/train_images_png/10006/1864590858.png,3,59.153744,0.021652,0.059143,0.040283,0.048492,0.045959
3,2,10006,1874946579,1,1,61.0,0,0,0,NaN,0,NaN,29,False,R-CC,input_data/train_images/10006/1874946579.dcm,input_data/train_images_png/10006/1874946579.png,3,59.153744,0.017319,0.033752,0.041626,0.023788,0.033722
4,2,10011,220375232,0,1,55.0,0,0,0,0.0,0,NaN,21,True,L-CC,input_data/train_images/10011/220375232.dcm,input_data/train_images_png/10011/220375232.png,1,60.198516,0.186523,0.044769,0.016220,0.005600,0.027374


In [64]:
df['mean_pred'] =df[['fold_1_predictions', 'fold_2_predictions', 'fold_3_predictions','fold_4_predictions']].mean(axis=1)

In [65]:
metrics.roc_auc_score( df['cancer'].values,df['mean_pred'].values)

0.8934869485166763

In [69]:
metrics.confusion_matrix(df['cancer'].values,np.where(df['mean_pred'].values>0.5,1,0))

array([[53257,   291],
       [  924,   234]])

In [27]:
def pfbeta(labels, predictions, beta=1.):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / max(y_true_count, 1)  # avoid / 0
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

def optimal_f1(labels, predictions):
    labels = labels.flatten()
    thres = np.linspace(0, 1, 101)
    f1s = [pfbeta(labels, predictions > thr) for thr in thres]
    idx = np.argmax(f1s)
    return f1s[idx], thres[idx]

In [68]:
optimal_f1(df['cancer'].values,df['mean_pred'].values)

(0.3486387647297846, 0.35000000000000003)

In [96]:
pfbeta(df['cancer'].values,np.where(df['mean_pred'].values>0.5,1,0))

0.27807486631016043

In [25]:
# df.to_csv('output/puesdolabelling.csv',index=False)

In [5]:
df = pd.read_csv('output/puesdolabelling.csv')

In [7]:
df['fold_1_predictions'].values


array([0.04346, 0.05054, 0.05914, ..., 0.033  , 0.02661, 0.01175])

In [8]:
np.save(''.join(
                'output/'+"_ddp_ep_"+"_"+"predictions.npy"), df['fold_1_predictions'].values)

In [149]:
df['mean_pred'] =df[['fold_1_predictions', 'fold_2_predictions', 'fold_3_predictions','fold_4_predictions']].mean(axis=1)

In [110]:
df.columns

Index(['site_id', 'patient_id', 'image_id', 'laterality', 'view', 'age',
       'cancer', 'biopsy', 'invasive', 'BIRADS', 'implant', 'density',
       'machine_id', 'difficult_negative_case', 'orientation', 'orignal_path',
       'dump_path', 'kfold', 'mean_age', 'fold_0_predictions',
       'fold_1_predictions', 'fold_2_predictions', 'fold_3_predictions',
       'fold_4_predictions'],
      dtype='object')

In [154]:
pfbeta(df[~df.kfold.isin([0,1])]['cancer'].values,np.where(df[~df.kfold.isin([0,1])]['mean_pred'].values>0.3,1,0))

0.32788671023965144

In [155]:
pfbeta(df[df.kfold.isin([1])]['cancer'].values,np.where(df[df.kfold.isin([1])]['mean_pred'].values>0.3,1,0))

0.32996632996632996

In [111]:
cols_to_train = features + ['fold_1_predictions', 'fold_2_predictions', 'fold_3_predictions','fold_4_predictions'] + ['kfold'] + ['cancer']

In [112]:
ml_features = features + ['fold_1_predictions', 'fold_2_predictions', 'fold_3_predictions','fold_4_predictions']

In [113]:
df_ml = df[cols_to_train]

In [114]:
df_ml.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,fold_1_predictions,fold_2_predictions,fold_3_predictions,fold_4_predictions,kfold,cancer
0,2,10006,462822612,0,1,61.0,0,29,0.04346,0.04540,0.05010,0.11660,3,0
1,2,10006,1459541791,0,0,61.0,0,29,0.05054,0.04614,0.01990,0.03140,3,0
2,2,10006,1864590858,1,0,61.0,0,29,0.05914,0.04028,0.04850,0.04596,3,0
3,2,10006,1874946579,1,1,61.0,0,29,0.03375,0.04163,0.02379,0.03372,3,0
4,2,10011,220375232,0,1,55.0,0,21,0.04477,0.01622,0.00560,0.02737,1,0


In [115]:
df_test_ml = df_ml[df_ml.kfold==1].reset_index(drop=True)
x_test = df_ml[df_ml.kfold==0].reset_index(drop=True)
df_ml = df_ml[~df_ml.kfold.isin([0,1])].reset_index(drop=True)

In [116]:
df_ml.kfold.unique()

array([3, 4, 2])

In [117]:
df_test_ml.shape, x_test.shape , df_ml.shape

((10950, 14), (10934, 14), (32822, 14))

In [120]:
y_train = df_ml[target]
y_test = x_test[target]

In [141]:
final_y = df_test_ml[target]

In [142]:
y_train.shape, y_test.shape , final_y.shape

((32822, 1), (10934, 1), (10950, 1))

# Stacking

In [15]:
ntrain = df_ml.shape[0]
ntest = x_test.shape[0]
base_test = df_ml.shape[0]
SEED = 1993 # for reproducibility\
NFOLDS = df_ml.kfold.unique() #set folds for out-of-fold prediction

In [16]:
ntrain , ntest

(32822, 10934)

In [37]:
# opening the file- model_jlib
# imputer_jlib = joblib.load('imputer')
# df_test_infer = df_test[df_test.columns.difference(['ID','Output'])]

In [82]:
def get_oof(clf, df, x_test,final_test,model=None,params=None,categorical_features=None):
    oof_train = np.zeros((df.shape[0],))
    oof_test = np.zeros((ntest,))
    test =  np.empty((len(NFOLDS),final_test.shape[0]))
    oof_test_skf = np.empty((len(NFOLDS), ntest))
    final_test = final_test[ml_features]
    clf = clf(**params)
        
        
    if categorical_features is not None:
        cat_idx = np.where(x_train.columns.isin(categorical_features))[0]
        x_train[categorical_features] =  x_train[categorical_features].astype(int)
        x_test[categorical_features] =  x_test[categorical_features].astype(int)
        final_test[categorical_features] =  final_test[categorical_features].astype(int)
    
    for i ,fold in enumerate(NFOLDS):
        # test- training
        train_set, valid_set = df[df['kfold']!= fold], df[df['kfold'] == fold]
        x_tr,x_te  = train_set[ml_features], valid_set[ml_features]
        y_tr,y_te = train_set[target], valid_set[target]
    
            
        if model == 'lightgbm':
            print(f'{model} : Running !')
            clf.fit(x_tr,y_tr,eval_set=[(x_te,y_te)],early_stopping_rounds=300,verbose=100)
            
        elif model == 'catboost':
            print(f'{model} : Running !')
            clf.fit(x_tr,y_tr,eval_set=(x_te,y_te),early_stopping_rounds=300,verbose=50)
        elif model == 'xgboost':
            print(f'{model} : Running !')
            clf.fit(x_tr,y_tr,eval_set=[(x_te,y_te)],early_stopping_rounds=300,verbose=50)
        else:
            print(f'{model} : Running !')
            clf.fit(x_tr, y_tr)


        oof_train[x_te.index] = clf.predict(x_te).ravel()
        print("AUC-ROC score is ",metrics.roc_auc_score(y_te,clf.predict(x_te)))
        if model == 'catboost':
            print("pfbeta score", pfbeta(y_te.values,clf.predict(x_te)))
            print("OPtimal F1 ",  optimal_f1(y_te.values, clf.predict_proba(x_te)[:,1]))
        else:
            print("pfbeta score", pfbeta(y_te.values,np.where(clf.predict(x_te)>0.5,1,0)))
            print("OPtimal F1 ",  optimal_f1(y_te.values, clf.predict(x_te)))

        oof_test_skf[i, :] = clf.predict(x_test).ravel()
        test[i, :] = clf.predict(final_test).ravel()

    oof_test[:] = oof_test_skf.mean(axis=0)
    test = test.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1), test 

In [87]:
catboost_params = {'n_estimators':3000,'random_state' :1993,'eval_metric':'AUC','verbose' : 100,'task_type':'CPU','early_stopping_rounds':300}

lgb_params = { 
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc',
              'learning_rate' : 0.03,
              'num_boost_round':2000,
#               'max_depth' : 12, 
                'seed': 7,
                'verbose': 100
               }

xg_params = {'train_method':'hist','nthread':4,'eval_metric':'auc','eta':0.05,'seed':1993,'objective':'binary:logistic','grow_policy':'lossguide'} 


rf_params = { 'n_jobs': -1,'n_estimators': 1000,'criterion': 'entropy'} #'max_depth': 12



In [73]:
lgb_oof_train, lgb_oof_test ,lgb_test  = get_oof(LGBMClassifier,df_ml.reset_index(drop=True), x_test[ml_features],df_test_ml[ml_features],model="lightgbm",params=lgb_params) 

lightgbm : Running !
[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Info] Number of positive: 464, number of negative: 21419
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.165048
[LightGBM] [Debug] init for col-wise cost 0.400616 seconds, init for row-wise cost 2.787687 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.901080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1615
[LightGBM] [Info] Number of data points in the train set: 21883, number of used features: 12
[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.021204 -> initscore=-3.832149
[LightGBM] [Info] Start training from score -3.832149
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 

In [64]:
cb_oof_train, cb_oof_test,cb_test = get_oof(CatBoostClassifier, df_ml, x_test[ml_features], df_test_ml[ml_features], model="catboost", params=catboost_params) 

catboost : Running !
Learning rate set to 0.042108
0:	test: 0.6291463	best: 0.6291463 (0)	total: 4.72ms	remaining: 14.1s
50:	test: 0.8256800	best: 0.8332238 (19)	total: 604ms	remaining: 34.9s
100:	test: 0.8123738	best: 0.8332238 (19)	total: 910ms	remaining: 26.1s
150:	test: 0.8059141	best: 0.8332238 (19)	total: 1.29s	remaining: 24.3s
200:	test: 0.8028493	best: 0.8332238 (19)	total: 1.71s	remaining: 23.8s
250:	test: 0.7996118	best: 0.8332238 (19)	total: 2.2s	remaining: 24.1s
300:	test: 0.7953820	best: 0.8332238 (19)	total: 2.57s	remaining: 23.1s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8332238336
bestIteration = 19

Shrink model to first 20 iterations.
AUC-ROC score is  0.5147126195238049
pfbeta score 0.056680161943319846
OPtimal F1  (0.21220930232558138, 0.14)
catboost : Running !
Learning rate set to 0.042107
0:	test: 0.8561369	best: 0.8561369 (0)	total: 5.04ms	remaining: 15.1s
50:	test: 0.9106930	best: 0.9108157 (47)	total: 302ms	remaining: 17.4s
100:	test

In [67]:
xg_oof_train, xg_oof_test,xg_test= get_oof(XGBClassifier, df_ml, x_test[ml_features], df_test_ml[ml_features],model="xgboost",params=xg_params) 

xgboost : Running !
[01:17:26] WARNING: ../src/learner.cc:767: 
Parameters: { "train_method" } are not used.

[0]	validation_0-auc:0.65027
[50]	validation_0-auc:0.75272
[99]	validation_0-auc:0.77390
AUC-ROC score is  0.5439977633066347
pfbeta score 0.14999999999999997
OPtimal F1  (0.14999999999999997, 0.0)
xgboost : Running !
[01:18:19] WARNING: ../src/learner.cc:767: 
Parameters: { "train_method" } are not used.

[0]	validation_0-auc:0.83409
[50]	validation_0-auc:0.89071
[99]	validation_0-auc:0.89504
AUC-ROC score is  0.5595044914517531
pfbeta score 0.2014388489208633
OPtimal F1  (0.2014388489208633, 0.0)
xgboost : Running !
[01:19:10] WARNING: ../src/learner.cc:767: 
Parameters: { "train_method" } are not used.

[0]	validation_0-auc:0.80693
[50]	validation_0-auc:0.88399
[99]	validation_0-auc:0.87827
AUC-ROC score is  0.5488219303132074
pfbeta score 0.16974169741697417
OPtimal F1  (0.16974169741697417, 0.0)


In [74]:
df_ml.fillna(method="ffill",inplace=True)

In [81]:
df_test_ml.fillna(method="ffill",inplace=True)

In [88]:
rf_oof_train, rf_oof_test ,rf_test = get_oof(RandomForestClassifier, df_ml, x_test[ml_features], df_test_ml[ml_features], model="randomforest",params=rf_params)

randomforest : Running !
AUC-ROC score is  0.535330375229869
pfbeta score 0.1227436823104693
OPtimal F1  (0.1227436823104693, 0.0)
randomforest : Running !
AUC-ROC score is  0.5491021121092114
pfbeta score 0.17358490566037738
OPtimal F1  (0.17358490566037738, 0.0)
randomforest : Running !
AUC-ROC score is  0.5383728961460068
pfbeta score 0.138996138996139
OPtimal F1  (0.138996138996139, 0.0)


### Meta Model

In [89]:
#### Model 2 :
base_predictions_train = pd.DataFrame( {
    'lightgbm': lgb_oof_train.ravel(),
     'catboost': cb_oof_train.ravel(),
     'xgboost': xg_oof_train.ravel(),
     'randomforest':rf_oof_train.ravel()
                                  
    })

In [90]:
corr = base_predictions_train.corr()
corr.style.background_gradient(cmap='coolwarm')

,lightgbm,catboost,xgboost,randomforest
lightgbm,1.000000,0.265149,0.484136,0.494337
catboost,0.265149,1.000000,0.662823,0.704222
xgboost,0.484136,0.662823,1.000000,0.743763
randomforest,0.494337,0.704222,0.743763,1.000000


In [91]:
base_test =pd.DataFrame( {
    'lightgbm': lgb_oof_test.ravel(),
     'catboost': cb_oof_test.ravel(),
     'xgboost': xg_oof_test.ravel(),
     'randomforest': rf_oof_test.ravel(),
    #  'linearregression':lr_oof_test.ravel()                     
    })

In [92]:
final_pred =pd.DataFrame( {
    'lightgbm': lgb_test.ravel(),
     'catboost': cb_test.ravel(),
     'xgboost': xg_test.ravel(),
     'randomforest': rf_test.ravel(),
    #  'linearregression':lr_test.ravel()                     
    })

In [102]:
base_predictions_train.shape ,predictions_model_level1.shape 

((32822, 4), (10934,), (10950, 1))

In [122]:
clf_lr = XGBClassifier(random_state=0).fit(base_predictions_train, y_train)
predictions_model_level1=clf_lr.predict(base_test)

In [123]:
metrics.confusion_matrix(y_test,predictions_model_level1) 

array([[10689,    14],
       [  199,    32]])

In [131]:
optimal_f1(y_test.values,clf_lr.predict_proba(base_test)[:,1])

(0.2913907284768212, 0.02)

In [136]:
print("pfbeta score", pfbeta(y_test.values,np.where(clf_lr.predict_proba(base_test)[:,1]>0.3,1,0)))

pfbeta score 0.28762541806020064


In [137]:
all_pred = clf_lr.predict(final_pred)

In [146]:
print("pfbeta score", pfbeta(final_y.values,np.where(clf_lr.predict_proba(final_pred)[:,1]>0.3,1,0)))

pfbeta score 0.27586206896551724


In [147]:
final_pred

,lightgbm,catboost,xgboost,randomforest
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
10945,0.0,0.0,0.0,0.0
10946,0.0,0.0,0.0,0.0
10947,0.0,0.0,0.0,0.0
10948,0.0,0.0,0.0,0.0


## Final Prediction

In [267]:
## Inference
for_testing = df_test.copy()
for_testing['Prediction'] = all_pred


In [268]:
for_testing.tail()

,ID,Feature 1,Feature 2,Feature 3,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Output,Feature 4_E,Feature 4_ENE,Feature 4_N,Feature 4_NE,Feature 4_NNE,Feature 4_NNW,Feature 4_NW,Feature 4_S,Feature 4_SE,Feature 4_SSE,Feature 4_SSW,Feature 4_SW,Feature 4_W,Feature 4_WNW,Prediction
460,461.0,143405.8,2641337.88,0.6,38640.0,541887360.0,4.0,4.0,311.0,11.0,5.0,3.0,30.0,4.0,8.206000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.288715
419,420.0,137335.8,2646582.88,1.2,8034.0,59001696.0,3.0,2.0,234.0,8.0,4.0,3.0,26.0,3.0,8.340327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.421587
92,93.0,142697.8,2640266.88,5.0,20271.0,873659829.0,18.0,12.0,1218.0,29.0,14.0,15.0,87.0,12.0,104.427370,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.166473
163,164.0,130222.8,2640677.88,4.2,28910.0,793434950.0,1.0,5.0,491.0,17.0,7.0,1.0,35.0,1.0,9.870458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.766466
360,361.0,107381.8,2626811.88,2.0,35038.5,346459568.0,3.0,2.0,338.0,6.0,4.0,2.0,26.0,3.0,8.501699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.595958


In [269]:
r2_score(for_testing['Output'],for_testing['Prediction']), mean_squared_error(for_testing['Output'],for_testing['Prediction']) , mean_absolute_error(for_testing['Output'],for_testing['Prediction'])

(0.9906307467944304, 25.90092679363326, 2.0555595019427777)

# NN

In [361]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    

In [362]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [363]:
x_train.head()

,Feature 1,Feature 2,Feature 3,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 4_E,Feature 4_ENE,Feature 4_N,Feature 4_NE,Feature 4_NNE,Feature 4_NNW,Feature 4_NW,Feature 4_S,Feature 4_SE,Feature 4_SSE,Feature 4_SSW,Feature 4_SW,Feature 4_W,Feature 4_WNW
242,106897.8,2620112.88,3.5,31788.0,817364844.0,7.0,5.0,324.0,18.0,6.0,5.0,36.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
401,110013.8,2629221.38,1.4,33775.0,489399750.0,9.0,3.0,375.0,6.0,4.0,5.0,25.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
305,125833.8,2631701.88,2.7,39800.0,274739400.0,15.0,4.0,481.0,11.0,6.0,13.0,40.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
317,113633.8,2624857.88,2.5,21924.0,363675312.0,1.0,3.0,435.0,9.0,5.0,11.8,29.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
507,133780.8,2645980.88,5.7,7848.0,77663808.0,1.0,3.0,264.0,6.0,4.0,1.0,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [364]:
EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.02

In [365]:
## train data
class trainData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.x_data)



## test data    
class testData(Dataset):
    
    def __init__(self, x_data):
        self.x_data = x_data
        
    def __getitem__(self, index):
        return self.x_data[index]
        
    def __len__ (self):
        return len(self.x_data)
    


In [366]:
train_data = trainData(torch.FloatTensor(x_train.values), torch.FloatTensor(y_train.values))
test_data = testData(torch.FloatTensor(x_test.values))

In [367]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [368]:
class Regressor_Model(nn.Module):
    def __init__(self,input_dims):
        super(Regressor_Model, self).__init__()
        self.layer_1 = nn.Linear(input_dims, 32) 
        self.layer_2 = nn.Linear(32, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(32)
        
    def forward(self, inputs,targets=None):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        
        if targets is not None: 
            loss = nn.MSELoss()(x, targets.unsqueeze(1))
            output = x.detach().cpu().numpy()

            targets = targets.detach().cpu().numpy()

            mse =mean_squared_error( targets,output) 
            return x, loss, mse
        
        
        return x,  None, None

In [369]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.0001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(
                "EarlyStopping counter: {} out of {}".format(
                    self.counter, self.patience
                )
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print("Validation score improved ({} --> {}). Saving model!".format(
                        self.val_score, epoch_score
                    )
                )
            
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [370]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [445]:
class Engine:
    @staticmethod
    def train(
        data_loader,
        model,
        optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        fp16=False,
    ):

        losses = AverageMeter()
        accuracies = AverageMeter()
        final_predictions = []
        model.train()

        if fp16:
            scaler = torch.cuda.amp.GradScaler()    

        for b_idx, data in enumerate(data_loader):
            
            X_batch, y_batch = data
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    
            if accumulation_steps == 1 and b_idx == 0:
                optimizer.zero_grad()
            if fp16:    
                with torch.cuda.amp.autocast():    
                    predictions, loss, accuracy = model(X_batch,y_batch)
            else:
                predictions, loss, accuracy = model(X_batch,y_batch)

            predictions = predictions.detach().cpu().numpy()  
            final_predictions.append(predictions)
            with torch.set_grad_enabled(True):
                if fp16:
                    scaler.scale(loss).backward()                   
                else:
                    loss.backward()
                if (b_idx + 1) % accumulation_steps == 0:
                    if fp16:
                        scaler.step(optimizer)
                        scaler.update()
                    else:     
                        optimizer.step()
                    if scheduler is not None:
                         scheduler.step()
                    if b_idx > 0:
                        optimizer.zero_grad()

                losses.update(loss.item(), data_loader.batch_size)
                accuracies.update(accuracy.item(), data_loader.batch_size)

        return final_predictions, losses.avg, accuracies.avg

    @staticmethod
    def evaluate(data_loader, model, device):
        losses = AverageMeter()
        accuracies = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            for b_idx, data in enumerate(data_loader):    
                X_batch,y_batch = data
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                predictions, loss, accuracy = model(X_batch,y_batch)
                predictions = predictions.detach().cpu().numpy()  
                
                final_predictions.append(predictions) 
                
                losses.update(loss.item(), data_loader.batch_size)    
                accuracies.update(accuracy.item(), data_loader.batch_size)

        return final_predictions, losses.avg, accuracies.avg

    @staticmethod
    def predict(data_loader, model, device):
        model.eval()
        final_predictions = []

        with torch.no_grad():

            for b_idx, data in enumerate(data_loader):    
                X_batch = data
                X_batch = X_batch.to(device)
                predictions, _, _ = model(X_batch)
                predictions = predictions.detach().cpu().numpy()  
                print(predictions,">>>>>")
                final_predictions.append(predictions) 
                   
        return final_predictions

In [470]:
def run_training(X_train,y_train, X_test,y_test,final_test):
    
    epochs = 500
    train_bsize = BATCH_SIZE
    valid_bsize = 24
    device= "cpu"
    n_patience = 50
    
    model = Regressor_Model(input_dims= X_train.shape[1])
    model.to(device)
    
    train_data = trainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train.values))
    test_data = trainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test.values))
    
    train_loader = DataLoader(dataset=train_data, batch_size=train_bsize, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=valid_bsize)
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=epochs, T_mult=1, eta_min=1e-6, last_epoch=-1)
    
    es = EarlyStopping(patience=n_patience, mode="min")
    
    
    
    history = pd.DataFrame(columns=['train_loss','val_loss','train_mse','val_mse'])
    
    # TRAIN
    print('Training...')
    for epoch in range(epochs):
        
        for param_group in optimizer.param_groups:
            lr = param_group['lr']

        predictions, train_loss, train_accuracy = Engine.train(train_loader, model, optimizer, device=device)
        train_predictions = np.vstack((predictions))
        predictions, valid_loss, val_accuracy = Engine.evaluate(test_loader, model, device=device)
        
        predictions = np.vstack((predictions))
        
        history = history.append({'train_loss':train_loss,'val_loss':valid_loss,'train_mse':train_accuracy,'val_mse':val_accuracy}, ignore_index=True)
        print(f"Epoch {epoch+1:03}: | train_loss: {train_loss:.4f} | val_loss {valid_loss:.4f} | train_mse {train_accuracy:.4f} | val_mse {val_accuracy:.4f} | lr {lr:.6f} ")

        
        scheduler.step(epoch) 

        es(val_accuracy, model, model_path=f"model.bin")
        
        if es.counter == 0: 
            best_prediction = predictions
        if es.early_stop:
            print("Early stopping")
            break
    
    
    valid_data = testData(torch.FloatTensor(final_test))
    valid_loader = DataLoader(dataset=valid_data, batch_size=valid_bsize)
    model_path=f"model.bin"
    valid_bsize = 24
    
    model = Regressor_Model(input_dims= final_test.shape[1])
    model.load_state_dict(torch.load(model_path))
    model.to(device)

    # PREDICT
    print('Predicting...')
    predictions = Engine.predict(valid_loader, model, device=device)
    print(predictions,">>>>>>")
    final_preds = None
    for p in predictions:
        if final_preds is None:
            final_preds = p
        else:
            final_preds = np.vstack((final_preds, p))
    
    return history  , final_preds

In [469]:
df_test_infer_standerdized = scaler.transform(df_test_infer[x_train.columns])

In [471]:
data_history, nn_final_prediction = run_training(X_train,y_train, X_test,y_test,df_test_infer_standerdized)

Training...
Epoch 001: | train_loss: 3612.9588 | val_loss 2424.8119 | train_mse 3612.9588 | val_mse 2424.8119 | lr 0.020000 
Validation score improved (inf --> 2424.8118896484375). Saving model!
Epoch 002: | train_loss: 2379.1756 | val_loss 1607.0671 | train_mse 2379.1755 | val_mse 1607.0673 | lr 0.020000 
Validation score improved (2424.8118896484375 --> 1607.0673014322917). Saving model!
Epoch 003: | train_loss: 1071.6183 | val_loss 774.3820 | train_mse 1071.6183 | val_mse 774.3820 | lr 0.020000 
Validation score improved (1607.0673014322917 --> 774.3819783528646). Saving model!
Epoch 004: | train_loss: 394.3356 | val_loss 294.9667 | train_mse 394.3356 | val_mse 294.9667 | lr 0.019999 
Validation score improved (774.3819783528646 --> 294.9666748046875). Saving model!
Epoch 005: | train_loss: 329.0922 | val_loss 293.5644 | train_mse 329.0922 | val_mse 293.5644 | lr 0.019998 
Validation score improved (294.9666748046875 --> 293.5643882751465). Saving model!
Epoch 006: | train_loss: 252

In [410]:
data_history

,train_loss,val_loss,train_mse,val_mse
0,3832.679171,2625.731608,3832.679058,2625.731364
1,3385.044481,3345.418538,3385.044443,3345.418538
2,2735.124211,3312.131999,2735.124268,3312.131999
3,2258.631592,1777.636475,2258.631658,1777.636434
4,2062.093637,1748.232259,2062.093665,1748.232218
...,...,...,...,...
84,1969.189707,1720.276815,1969.189632,1720.276774
85,2112.359103,1809.143595,2112.359103,1809.143717
86,1932.577627,1778.291626,1932.577599,1778.291545
87,2022.841797,1770.910767,2022.841759,1770.910522


In [472]:
for_testing['NN_Predictions'] = nn_final_prediction.ravel()

In [473]:
for_testing.head()

,ID,Feature 1,Feature 2,Feature 3,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Output,Feature 4_E,Feature 4_ENE,Feature 4_N,Feature 4_NE,Feature 4_NNE,Feature 4_NNW,Feature 4_NW,Feature 4_S,Feature 4_SE,Feature 4_SSE,Feature 4_SSW,Feature 4_SW,Feature 4_W,Feature 4_WNW,Prediction,NN_Predictions
56,57.0,99934.8,2627714.88,2.1,21014.0,932202054.0,10.0,13.0,711.0,18.7,13.0,6.0,79.0,5.0,136.582405,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.849785,121.227264
400,401.0,126709.8,2636825.88,1.4,29733.0,685434849.0,6.0,5.0,264.0,11.0,5.0,5.0,33.0,4.0,8.390682,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.421587,8.343574
227,228.0,106518.8,2619915.38,3.6,28086.0,791323050.0,5.0,4.0,372.0,15.0,5.0,4.0,31.0,3.0,9.219234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.334773,10.230247
549,550.0,142562.8,2622507.88,5.1,16400.0,281981600.0,9.0,2.0,161.0,6.0,3.0,8.0,20.0,8.0,4.851797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.019100,6.183632
286,287.0,128230.8,2643220.38,3.0,29342.0,777592342.0,8.0,3.0,442.0,11.0,6.0,7.0,34.0,5.0,8.892855,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.971122,11.418473


In [474]:
r2_score(for_testing['Output'],for_testing['NN_Predictions']), mean_squared_error(for_testing['Output'],for_testing['NN_Predictions']) , mean_absolute_error(for_testing['Output'],for_testing['NN_Predictions'])

(0.9558012111572417, 122.18578888475528, 5.173877119751323)